In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import torch 

ipl = pd.read_csv('ipl_data.csv')
ipl.tail(15)

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
75999,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,105,7,17.6,29,2,28,10,129
76000,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,MG Johnson,JD Unadkat,106,7,18.1,28,2,28,11,129
76001,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,JD Unadkat,106,7,18.2,28,1,28,11,129
76002,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,JD Unadkat,107,7,18.3,29,1,29,11,129
76003,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,MG Johnson,JD Unadkat,108,7,18.4,30,1,29,12,129
76004,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,JD Unadkat,114,7,18.5,36,1,35,12,129
76005,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,JD Unadkat,115,7,18.6,36,1,36,12,129
76006,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,116,7,19.0,37,0,36,12,129
76007,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,116,7,19.1,37,0,36,12,129
76008,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,120,7,19.2,40,0,40,12,129


In [13]:
#Dropping certain features 
df = ipl.drop(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5','mid', 'striker', 'non-striker'], axis =1)
df.head()

,venue,bat_team,bowl_team,batsman,bowler,total
0,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,222
1,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,222
2,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,222
3,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,222
4,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,222


In [41]:
X = df.drop(['total'], axis =1)
y = df['total']

In [15]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object for each categorical feature
venue_encoder = LabelEncoder()
batting_team_encoder = LabelEncoder()
bowling_team_encoder = LabelEncoder()
striker_encoder = LabelEncoder()
bowler_encoder = LabelEncoder()

# Fit and transform the categorical features with label encoding
X['venue'] = venue_encoder.fit_transform(X['venue'])
X['bat_team'] = batting_team_encoder.fit_transform(X['bat_team'])
X['bowl_team'] = bowling_team_encoder.fit_transform(X['bowl_team'])
X['batsman'] = striker_encoder.fit_transform(X['batsman'])
X['bowler'] = bowler_encoder.fit_transform(X['bowler'])

In [22]:
# Train test Split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)

(53209, 5)
(53209,)


In [43]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Huber loss with delta parameter
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta

    def forward(self, y_pred, y_true):
        loss = torch.where(torch.abs(y_pred - y_true) < self.delta, 0.5 * (y_pred - y_true)**2, self.delta * (torch.abs(y_pred - y_true) - 0.5 * self.delta))
        return torch.mean(loss)

# Define the neural network model
class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 618)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(618, 158)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(158, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the model
input_size = X_train_scaled.shape[1]
model = MyModel(input_size)

# Define the Huber loss with delta=1.0
huber_loss = HuberLoss(delta=1.0)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print the model architecture
print(model)

MyModel(
  (fc1): Linear(in_features=5, out_features=618, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=618, out_features=158, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=158, out_features=1, bias=True)
)


In [39]:
# Assuming you have your data in PyTorch tensors X_train_tensor and y_train_tensor

# Convert your data to PyTorch tensors if they are not already
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Assuming y_train is a 1D array

# Set up the number of epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(X_train_tensor)
    
    # Compute the loss
    loss = huber_loss(y_pred, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 157.5995
Epoch [20/100], Loss: 152.2291
Epoch [30/100], Loss: 143.0647
Epoch [40/100], Loss: 128.0168
Epoch [50/100], Loss: 104.2603
Epoch [60/100], Loss: 68.7542
Epoch [70/100], Loss: 33.1840
Epoch [80/100], Loss: 34.7370
Epoch [90/100], Loss: 30.8227
Epoch [100/100], Loss: 30.1628


In [31]:
# Assuming you have your test data in pandas Series X_test_scaled and y_test

# Convert your test data to PyTorch tensors if they are not already
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)  # Assuming y_test is a 1D array

# Set the model to evaluation mode (important for layers like dropout)
model.eval()

# Make predictions on the test data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Calculate Mean Absolute Error (MAE)
mae = torch.mean(torch.abs(y_pred - y_test_tensor)).item()

print(f'Mean Absolute Error on Test Data: {mae:.4f}')

Mean Absolute Error on Test Data: 29.9291


In [35]:
import ipywidgets as widgets
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings("ignore")

venue = widgets.Dropdown(options=df['venue'].unique().tolist(), description='Select Venue:')
batting_team = widgets.Dropdown(options=df['bat_team'].unique().tolist(), description='Select Batting Team:')
bowling_team = widgets.Dropdown(options=df['bowl_team'].unique().tolist(), description='Select Batting Team:')
striker = widgets.Dropdown(options=df['batsman'].unique().tolist(), description='Select Striker:')
bowler = widgets.Dropdown(options=df['bowler'].unique().tolist(), description='Select Bowler:')

predict_button = widgets.Button(description="Predict Score")

def predict_score(b):
    with output:
        clear_output()  # Clear the previous output

        # Decode the encoded values back to their original values
        decoded_venue = venue_encoder.transform([venue.value])
        decoded_batting_team = batting_team_encoder.transform([batting_team.value])
        decoded_bowling_team = bowling_team_encoder.transform([bowling_team.value])
        decoded_striker = striker_encoder.transform([striker.value])
        decoded_bowler = bowler_encoder.transform([bowler.value])

        input = np.array([decoded_venue, decoded_batting_team, decoded_bowling_team, decoded_striker, decoded_bowler])
        input = input.reshape(1, 5)
        input = scaler.transform(input)

        # Convert the input to a PyTorch tensor
        input_tensor = torch.tensor(input, dtype=torch.float32)

        # Set the model to evaluation mode
        model.eval()

        # Make predictions on the input data
        with torch.no_grad():
            predicted_score_tensor = model(input_tensor)

        # Convert the predicted score tensor to a NumPy array
        predicted_score = int(predicted_score_tensor.item())

        print(predicted_score)

predict_button.on_click(predict_score)
output = widgets.Output()
display(venue, batting_team, bowling_team, striker, bowler, predict_button, output)

Dropdown(description='Select Venue:', options=('M Chinnaswamy Stadium', 'Punjab Cricket Association Stadium, M…

Dropdown(description='Select Batting Team:', options=('Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasth…

Dropdown(description='Select Batting Team:', options=('Royal Challengers Bangalore', 'Kings XI Punjab', 'Delhi…

Dropdown(description='Select Striker:', options=('SC Ganguly', 'BB McCullum', 'RT Ponting', 'DJ Hussey', 'Moha…

Dropdown(description='Select Bowler:', options=('P Kumar', 'Z Khan', 'AA Noffke', 'JH Kallis', 'SB Joshi', 'CL…

Button(description='Predict Score', style=ButtonStyle())

Output()